In [2]:
%pip install matplotlib


  Using cached cycler-0.12.1-py3-none-any.whl.metadata (3.8 kB)
  Using cached pillow-11.0.0-cp312-cp312-win_amd64.whl.metadata (9.3 kB)
  Using cached pyparsing-3.2.0-py3-none-any.whl.metadata (5.0 kB)
   ---------------------------------------- 0.0/7.8 MB ? eta -:--:--
   --------------------- ------------------ 4.2/7.8 MB 21.0 MB/s eta 0:00:01
   ---------------------------------------- 7.8/7.8 MB 19.4 MB/s eta 0:00:00
Using cached cycler-0.12.1-py3-none-any.whl (8.3 kB)
   ---------------------------------------- 0.0/2.2 MB ? eta -:--:--
   ---------------------------------------- 2.2/2.2 MB 13.8 MB/s eta 0:00:00
Using cached pillow-11.0.0-cp312-cp312-win_amd64.whl (2.6 MB)
Using cached pyparsing-3.2.0-py3-none-any.whl (106 kB)


In [4]:
import cv2
import numpy as np
#import matplotlib.pyplot as plt

In [5]:
def video_saver(cap, output_path):
    fps = cap.get(cv2.CAP_PROP_FPS)
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    fourcc = cv2.VideoWriter_fourcc(*'XVID')  # Example codec
    out = cv2.VideoWriter(output_path, fourcc, fps, (width,height))

    return out
    

In [ ]:
def VideoClassification(video_path, output_path):

    cap = cv2.VideoCapture(video_path)
    if not cap.isOpened():
        print("Error opening the video")
        exit()

    out = video_saver(cap, output_path)

    frame_cnt, eve_cnt, dusk_cnt, night_cnt = 0,0,0,0
    prediction = ""

    while True:
        ret, frame = cap.read()
        
        if not ret:
            print("End of video.")
            break

        frame_cnt += 1
        image_hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
        v = image_hsv[:, :, 2]
        
        eve_mask = (v > 110)
        dusk_mask = (v >= 30) & (v <= 70)
        night_mask = (v < 20)

        eve_p = np.sum(eve_mask)/v.size * 100
        dusk_p = np.sum(dusk_mask)/v.size * 100
        night_p = np.sum(night_mask)/v.size * 100

        if eve_p > dusk_p and eve_p > night_p:
            eve_cnt += 1
            prediction = "Evening"
        elif dusk_p > eve_p and dusk_p > night_p:
            dusk_cnt += 1
            prediction = "Dusk"
        else:
            night_cnt += 1
            prediction = "Night"

        text = f"Evening: {eve_p:.2f}%  |   Dusk: {dusk_p:.2f}% |   Night: {night_p:.2f}%"
        pred_text = f"Prediction: {prediction}"
        cv2.putText(img=frame, text=text, org=(20,50), fontFace=cv2.FONT_ITALIC, color = (255, 255, 255), fontScale=1)
        cv2.putText(img=frame, text=pred_text, org=(20,100), fontFace=cv2.FONT_ITALIC, color = (255, 255, 255), fontScale=1)
        
        frame_resized = cv2.resize(frame, (1024, 800), interpolation= cv2.INTER_LINEAR)

        cv2.imshow("video",frame_resized)
        out.write(frame)

        if cv2.waitKey(10) & 0xFF == ord('q'):
            break;

    cap.release()
    out.release()
    cv2.destroyAllWindows()
    
    print(f"Total no of frames: {frame_cnt}")
    per_split = {time_of_day : (cnt/frame_cnt)*100 for time_of_day, cnt in zip(["Evening", "Dusk", "Night"], [eve_cnt, dusk_cnt, night_cnt])}
    for time, per in per_split.items():
        print(f"{time} is {per:.2f}%")

    return

video_path = "E:/My_project/Day_Night/input/5763215-hd_1920_1080_30fps.mp4"
output_path = "E:/My_project/Day_Night/input/output/pred_video.mp4"


VideoClassification(video_path, output_path)


End of video.
Total no of frames: 1014
Evening is 88.36%
Dusk is 1.58%
Night is 10.06%
